# List Comprehension Syntax

* [Colab view](https://colab.research.google.com/github/4dsolutions/elite_school/blob/master/List_Comprehensions.ipynb)
* [nbviewer view](https://nbviewer.org/github/4dsolutions/elite_school/blob/master/List_Comprehensions.ipynb)
* [Repo](https://github.com/4dsolutions/elite_school/)

Pythonistas love this syntax, because it is compact and expressive i.e. "pythonic".

The idea is a for loop between square brackets generates the terms of the list.

```
    [<expression using var> for var in iterable]
```

is the core syntax.

Below we have chosen ```x``` to iterate over ```range(10)``` thereby binding the  name ```x``` to 0, 1, 2... 9.  Each time ```x``` binds to an object (in this case a number), the expression using ```x```, which is ```x * x```, computes the next term in the list.

In [1]:
L = [x * x for x in range(10)]

In [2]:
L

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In the code cell below, practice variations on ```[1]``` by:

* changing the variable name
* altering the expression
* changing the iterable (e.g. try a new range or maybe another list)

An optional if clause allows for filtering:

```
       [  <expression using var> 
       for var in iterable
       if <expression using var>  ]
```

Splitting a list comprehension across multiple lines may add readability but is never required.

In [3]:
words = ['tower', 'harry', 'potter', 'python']
M = [word.capitalize() 
     for word in words 
     if "p" in word]

In [4]:
M

['Potter', 'Python']

Practice makes perfect.  Consider adding some slice notation on the end, if you wish to practice a syntax we encountered earlier.

In [5]:
[w.capitalize() for w in ['hello', 'kitty']][::-1]  # adding slice notation

['Kitty', 'Hello']

Sometimes the expression does not need to involve any input from the for loop. Perhaps the new values are coming from someplace else, or maybe the expression is something static (unchanging).

Use of the name ```_``` typically signifies the values it takes on (binds to) have no relevance to the work at hand.  The point was simply to trigger repetition.  The for loop still runs multiple times, and sometimes that's all we need.

In [6]:
stars = [ "*" for _ in range(10)]

In [7]:
stars

['*', '*', '*', '*', '*', '*', '*', '*', '*', '*']

As our segue to a next topic, the generator function, check out the cells below.  Your instructor will explain, as we turn to this related aspect of Python.

Using ```print``` inside a list comprehension is not considered best practice, however doing it for instructive purposes may prove illuminating.  The list comprehension below prints as instructed, leaving now spaces between characters.  However, what ends up in the list is a sequence of None objects.  Why?  Because ```print()``` returns ```None``` while sending strings to the console (or other standard output device) as a side effect.

In [8]:
K = [print(c, end="") for c in "I like Python"]

I like Python

In [9]:
K

[None, None, None, None, None, None, None, None, None, None, None, None, None]

# Generator Functions

A generator function is quite like an ordinary function, except it depends on the keyword ```yield``` to hand a result back to the caller.  The generator suspends operations at this point, but remains ready to be nudged forward by the ```next``` function.  

When nudged to advance, the generator picks up right where it left off and continues executing.  Any local variables inside the generator remain intact, and may be updated.  In other words, generators "preserve state" between nudgings.

In [10]:
# Fibonacci Numbers
# https://oeis.org/A000045

def fibo(a=0, b=1):
    while True:
        yield a
        b, a = a + b, b
        
gen = fibo()
fibs = [next(gen) for _ in range(10)]
print(fibs)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


# Generator Expressions

A generator expression looks just like a list comprehension except it uses "curvy brackets" (i.e. parentheses) in place of square brackets.  The resulting object is a generator that computes next elements on demand (called "lazy evaluation" or "just in time"), instead of all at once, from beginning to end, the way a list does.

An advantage of generator expressions over list comprehensions is they usually require less memory.  Generators know how to get a next term, but they do not run on ahead unnecessarily, filling memory with terms a program my end up not needing.

In [11]:
fibs_gen = (next(gen) for _ in range(10))  # generator expression
type(fibs_gen)

generator

In [12]:
next(fibs_gen)

55

In [13]:
next(fibs_gen)

89

In [14]:
for n in fibs_gen:
    print(n, end=" ")

144 233 377 610 987 1597 2584 4181 

Keep in mind that generator expressions, like list comprehensions, are not designed to do a lot of complicated computing internally.  The syntax would get overwhelmed or at least hard to read. 

Even in the case of Fibonacci numbers, relatively light weight to compute, the task of coming up with the next one was delegated to the generator function named ```gen```.

Let's zoom out and look at the concept of "iterator" more generally.  If our computation is somewhat complicated, such as finding the next prime number, we still have the option to encapsulate it as a single object 

In [15]:
from primes import PrimeNumbers

In [16]:
for p in PrimeNumbers():
    print(p, end=" ")
    if p > 100:
        break 

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 

# Iterators

An iterator is any object that may be fed to ```next( )```.  They may at first seem like a small category of types, such as the just-introduced generator type.  However, any type that is an "iterable" (note the difference) may be turned into a "iterator" be feeding it to ```iter()```.  Once fed throught ```iter()```, the resulting object may be fed to ```next()```.

In [17]:
L = list(range(10))  # iterable
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [18]:
G = iter(L)  # create iterator named G

In [19]:
next(G)      # G may now be fed to next

0

In [20]:
next(G)

1

## Iterators as Classes

What the ```next()``` function is doing, behind the scenes, is accessing the ```__next__``` method of the target object.  Python's for-loop syntax is actually doing the same thing.

```python
    for var in L:
        print(var)
```

is technically the same as:

```python
    for var in iter(L):
        print(var)
```
   

In [21]:
for var in iter(L):
    print(var, end=" ")

0 1 2 3 4 5 6 7 8 9 

Python lets you define your own custom iterators.  Simply included two special name methods:  ```__next__``` and ```__iter__```.  An iterator should return itself if fed to ```iter``` and ```iter(obj)``` triggers ```obj.__iter__()``` just as ```next(obj)``` triggers ```obj.__next__()```.

In [22]:
class Fibo:
    
    def __init__(self, a, b):
        self.a, self.b = a, b
        
    def __next__(self):
        term = self.a
        # update for next time
        self.b, self.a = self.a + self.b, self.b
        return term  # not yield
    
    def __iter__(self):
        return self  # I am already an iterator

In [23]:
fibonacci = Fibo(0, 1)

In [24]:
[next(fibonacci) for _ in range(10)]

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

In [25]:
for var in Fibo(0, 1):
    if var < 100:
        print(var, end=" ")
    else:
        print(var)
        break

0 1 1 2 3 5 8 13 21 34 55 89 144


In [26]:
g = iter(Fibo(0,1))

In [27]:
g.__next__()

0

In [28]:
next(g)

1

In [29]:
g.__next__()

1